<a href="https://colab.research.google.com/github/Neiouo/Eric/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80_%E4%BD%9C%E6%A5%AD%E4%B8%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#1.生成資料

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 假設的學生名單與學科
students = [f"學生{i}" for i in range(1, 21)]
subjects = ["數學", "英文", "科學", "歷史", "地理"]

# 隨機生成學生資料
data = []

for student in students:
    for subject in subjects:
        # 每個學生每個學科的成績(0-100)
        grade = random.randint(60, 100)
        # 每個學生每個學科的觀看次數和完成次數
        watch_count = random.randint(0, 5)
        complete_count = random.randint(0, 5)
        # 隨機生成時間戳
        timestamp = datetime.now() - timedelta(days=random.randint(0, 30))

        # 添加數據
        data.append([student, subject, grade, watch_count, complete_count, timestamp])

# 創建 DataFrame
df = pd.DataFrame(data, columns=["actor.name", "theme", "grade", "觀看", "完成", "timestamp"])

# 顯示數據
df.head()


,actor.name,theme,grade,觀看,完成,timestamp
0,學生1,數學,99,1,4,2025-04-16 03:26:21.908362
1,學生1,英文,61,0,0,2025-03-25 03:26:21.908379
2,學生1,科學,69,4,5,2025-03-29 03:26:21.908387
3,學生1,歷史,74,0,5,2025-04-05 03:26:21.908394
4,學生1,地理,97,4,5,2025-04-17 03:26:21.908411


In [3]:
#2. 資料處理

# 提取時間信息
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['Date'] = df['timestamp'].dt.date
df['Time'] = df['timestamp'].dt.time
df['Weekday'] = df['timestamp'].dt.day_name()

# 顯示處理過的數據
df.head()


,actor.name,theme,grade,觀看,完成,timestamp,Date,Time,Weekday
0,學生1,數學,99,1,4,2025-04-16 03:26:21.908362,2025-04-16,03:26:21.908362,Wednesday
1,學生1,英文,61,0,0,2025-03-25 03:26:21.908379,2025-03-25,03:26:21.908379,Tuesday
2,學生1,科學,69,4,5,2025-03-29 03:26:21.908387,2025-03-29,03:26:21.908387,Saturday
3,學生1,歷史,74,0,5,2025-04-05 03:26:21.908394,2025-04-05,03:26:21.908394,Saturday
4,學生1,地理,97,4,5,2025-04-17 03:26:21.908411,2025-04-17,03:26:21.908411,Thursday


In [4]:
#3. 聚類分析 (KMeans)

from sklearn.cluster import KMeans

# 計算每個學生的總觀看次數和總完成次數
student_behavior_summary = df.groupby('actor.name').agg({'觀看': 'sum', '完成': 'sum'}).reset_index()

# 使用KMeans進行聚類
kmeans = KMeans(n_clusters=4, random_state=42)  # 假設分成4個群組
student_behavior_summary['cluster'] = kmeans.fit_predict(student_behavior_summary[['觀看', '完成']])

# 顯示聚類結果
student_behavior_summary.head()


,actor.name,觀看,完成,cluster
0,學生1,9,19,3
1,學生10,10,14,2
2,學生11,16,11,1
3,學生12,11,14,2
4,學生13,10,12,2


In [5]:
#4. 降維與視覺化 (PCA)

from sklearn.decomposition import PCA
import plotly.express as px

# 使用PCA進行降維
pca = PCA(n_components=2)
components = pca.fit_transform(student_behavior_summary[['觀看', '完成']])

# 添加PCA結果
student_behavior_summary['pca1'] = components[:, 0]
student_behavior_summary['pca2'] = components[:, 1]

# 視覺化聚類結果
fig = px.scatter(
    student_behavior_summary,
    x='pca1',
    y='pca2',
    color='cluster',
    title="學生行為聚類結果",
    labels={'pca1': '主成分1', 'pca2': '主成分2', 'cluster': '聚類群組'},
    hover_data=['actor.name', '觀看', '完成']
)

fig.update_layout(
    xaxis_title="主成分1",
    yaxis_title="主成分2",
    legend_title="群組",
    title_x=0.5,
    height=600,
    width=800
)

fig.show()

In [6]:
#5. 結果解釋

#PCA降維圖: 聚類結果會以不同顏色顯示，並顯示每個學生的行為特徵在2D空間中的分布。
#每個點代表一位學生，pca1 和 pca2 分別是主成分1和主成分2。
#學生行為群組: 聚類分析會根據學生的觀看次數和完成次數將學生分成幾個群組。
#例如，某些學生可能會集中在"觀看次數高，完成次數高"的區域，而其他學生則可能集中在"觀看次數低，完成次數低"的區域。

In [7]:
#6. 視覺化行為模式

# 只做題庫，不看影片
only_quiz = df[(df['觀看'] == 0) & (df['完成'] > 0)]

# 只看影片，不做題庫
only_watch = df[(df['觀看'] > 0) & (df['完成'] == 0)]

# 只進來晃一下，不看影片，也不做題庫
only_lurk = df[(df['觀看'] == 0) & (df['完成'] == 0)]

# 顯示不同學習行為模式的學生數量
summary = {
    '只做題庫': len(only_quiz['actor.name'].unique()),
    '只看影片': len(only_watch['actor.name'].unique()),
    '只進來晃一下': len(only_lurk['actor.name'].unique())
}

df_summary = pd.DataFrame(list(summary.items()), columns=['行為樣態', '學生數量'])

# 繪製條形圖
fig = px.bar(df_summary, x='行為樣態', y='學生數量', title="不同學習行為樣態的學生數量", labels={'學生數量': '學生數量'})
fig.show()


In [13]:
import pandas as pd
import plotly.graph_objects as go

# 假設 df 已經包含了 "actor.name", "theme", "觀看" 等字段
# 這裡是我們需要的數據透視表
pivot_subject_behavior = df.pivot_table(index='actor.name', columns='theme', values='觀看', fill_value=0)

# 顯示透視表的前幾行檢查數據
print(pivot_subject_behavior.head())
# 使用 Plotly 的熱力圖繪製
fig = go.Figure(data=go.Heatmap(
    z=pivot_subject_behavior.values,  # 熱力圖的數據矩陣
    x=pivot_subject_behavior.columns,  # 熱力圖的列標籤（學科）
    y=pivot_subject_behavior.index,  # 熱力圖的行標籤（學生）
    colorscale='YlGnBu'  # 顏色顯示方式，可以根據需要改變
))

# 設定圖表標題和軸標籤
fig.update_layout(
    title="不同學生在各科目觀看影片的次數",
    xaxis_title="科目",
    yaxis_title="學生",
    height=800,  # 設定圖表高度
    width=1000,  # 設定圖表寬度
)

# 顯示圖表
fig.show()


theme        地理   數學   歷史   科學   英文
actor.name                         
學生1         4.0  1.0  0.0  4.0  0.0
學生10        0.0  1.0  0.0  4.0  5.0
學生11        1.0  3.0  5.0  3.0  4.0
學生12        5.0  3.0  2.0  1.0  0.0
學生13        1.0  4.0  0.0  0.0  5.0
